In [26]:
try:
    import dimod
    from dwave.system import LeapHybridSampler
except: #this is needed or Colab will shit itself
    !pip install dimod

In [27]:
def LJ(s, e, r): #sigma, epsilon, radius (distance)
    dist_term = (r/s)**-6
    return 4 * e * (dist_term * (dist_term - 1)) #unreduced!

def point_distance(dim, a, b): #returns the distance (radius) between grid cell A and grid cell B
    return ((a//dim - b//dim)**2 + (a%dim - b%dim)**2)**0.5

def add_dict_values(a, b): #merges A and B - if A and B share keys, they will be added together.
    output = a
    for x in b:
        output[x] = output.get(x, 0) + b[x]
    return output

class problem_2D: #A wrapper that holds a potential graph and various misc. data, all in one trenchcoat.
    def __init__(self, dim):
        self.connections = {} #All the links between nodes, in form (nodeID1, nodeID2):strength
        self.positions = {} #all of the "physical" positions of the nodes, in form nodeID:(x,y). Usually this will simply be aligned with the grid, but might be staggered in future.
        self.dim = dim
        self.cellcount = dim**2
        for x in range(dim):
            for y in range(dim):
                self.positions[dim*y + x] = (x,y)

    def assignPotentials_LJ(self, sigma, epsilon, cutoff):
        for a in range(self.cellcount):
            for b in range(a+1, self.cellcount):
                distance = point_distance(self.dim,a,b)
                if(distance <= cutoff):
                    self.connections[(a,b)] = LJ(sigma, epsilon, distance)

    def assignPotentials_chemical(self, chemicalpotential):
        for x in range(self.cellcount):
            self.connections[(x,x)] = chemicalpotential

    def getBQM(self): #attempts to return a dimod.BQM object from this problem_2D.
        output = dimod.BinaryQuadraticModel(vartype='BINARY')
        for variable in self.positions:
            output.add_variable(variable)
        for connection in self.connections:
            a, b = connection
            if(a == b):
                output.add_linear(a, self.connections[connection])
            else:
                output.add_quadratic(a, b, self.connections[connection])
        return output
        return dimod.BinaryQuadraticModel(self.connections, {}, vartype='BINARY')

def quick_simple_BQM(dim, sigma, epsilon, chemicalpotential, cutoff): #quickly builds a simple BQM - no staggering, just a normal grid of nodes.
    output = problem_2D(dim)
    output.assignPotentials_chemical(chemicalpotential)
    output.assignPotentials_LJ(sigma, epsilon, cutoff)
    return output.getBQM()

def print_sample(sampledata): #Makes a pretty little thing to visualize a single SampleView object.
    dim = int((max(sampledata.keys()) + 1)**0.5)
    for y in range(dim):
        for x in range(dim):
            print(("░", "█")[sampledata[y*dim+x]], end='')
        print("")
def print_sampleset(sampleset): #same as above, but for an entire set.
    for sample in sampleset.samples():
        print_sample(sample)
        print("\n")


In [28]:
def sampleset_exact(BQM):
    sampler = dimod.ExactSolver().sample
    return sampler(BQM)

def sampleset_random(BQM, num_reads=10):
    sampler = dimod.RandomSampler().sample
    return sampler(BQM, num_reads=num_reads)

def sampleset_hybrid(BQM): #I pulled this from simulation.py, I have no idea how this works
    sampler = LeapHybridSampler()
    return sampler.sample(BQM)

In [29]:
def getlengths(sample): #Takes a single sample, returns a dict representing a histogram of all lengths.
    dim = int((max(sample.keys()) + 1)**0.5) #get the dimensions of the sample
    output = {}
    s = list(sample.values()) #extract dict values and make a list representing the microstate
    for a in range(len(s)):
        if(s[a] == 0): continue
        for b in range(a+1, len(s)):
          if(s[b] == 0): continue
          distance = point_distance(dim, a, b)
          output[distance] = output.get(distance, 0) + 1
    return output

def getlengths_sampleset(sampleset): #same as getlengths, except it works over a large sampleset instead.
    output = {}
    for sample in sampleset.samples():
        result = getlengths(sample)
        for x in result:
            output[x] = output.get(x, 0) + result[x] #add up all values in the two dicts
    return output

def print_dict_excel(x): #print in a way that can be copied into excel
    for item in x:
        print(item, x[item])

In [30]:
testresults = sampleset_hybrid(quick_simple_BQM(20,5.0,0.5,0.4,10))
print_sampleset(testresults)

print_dict_excel(getlengths_sampleset(testresults))
#todo - add proper statistics, like in CAP

░░░░░█░░░░░░░░░░░░░░
░░░░░░░░░░█░░░░░░░░░
░░░░░░░░░░░░░░█░░░░░
░░░░░░░░░░░░░░░░░░░░
░░░█░░░░░░░░░░░░░░░░
░░░░░░░░░░░█░░░░░░█░
░░░░░░░░░░░░░░░░░░░░
░░░░░░░█░░░░░░░█░░░░
░░█░░░░░░░░░░░░░░░░░
░░░░░░░░░░░█░░░░░░░█
░░░░░░░░░░░░░░░░░░░░
░░░░░█░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░█░░░░░░
░░░░░░░░░░░░░░░░░░░░
░░░░█░░░█░░░░░░░░░░░
█░░░░░░░░░░░░░░░░░░░
░░░░░░░░░░░█░░░░█░░░
░░░░░░░░░░░░░░░░░░░░
░░░░░░░░█░░░░░░░░░░█


5.0990195135927845 3
9.219544457292887 4
4.47213595499958 9
7.810249675906654 3
13.92838827718412 1
7.280109889280518 4
12.206555615733702 2
8.54400374531753 5
10.816653826391969 1
16.64331697709324 2
11.0 2
15.264337522473747 2
15.033296378372908 2
15.297058540778355 2
16.76305461424021 2
18.027756377319946 3
20.248456731316587 3
19.235384061671343 2
23.600847442411894 1
4.123105625617661 6
7.615773105863909 2
8.94427190999916 2
6.708203932499369 2
10.63014581273465 2
8.06225774829855 5
12.041594578792296 2
11.180339887498949 3
12.36931687685298 3
15.231546211727817 1
14